In [ ]:
!pip install PyMuPDF

import argparse
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import fitz  # PyMuPDF
import json
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/google_form_responses_database/code")

import configv3 as config # Import configuration from config.py
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_google_sheet_data(sheet_id, credentials_path):
    """Loads data from Google Sheets into a Pandas DataFrame."""
    try:
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_key(sheet_id)
        sheet = spreadsheet.sheet1
        data = sheet.get_all_records()
        df = pd.DataFrame(data)
        logging.info("Google Sheet data loaded successfully.")
        return df
    except Exception as e:
        logging.error(f"Failed to load Google Sheet data: {e}")
        return None

def fill_static_pdf(static_pdf_path, output_pdf_path, field_mapping, data):
    """Fills a static PDF form with data from a dictionary."""
    try:
        doc = fitz.open(static_pdf_path)

        for field_name in field_mapping.keys():
            key = field_mapping[field_name]['key']
            page_index = field_mapping[field_name]['page_index']
            x0, y0 = field_mapping[field_name]['position']
            value = data.get(key, '')  # Use .get() to handle missing keys gracefully

            if value is not None:
                page = doc[page_index]
                page.insert_text((x0, y0), str(value), fontsize=config.FONT_SIZE)

        doc.save(output_pdf_path)
        doc.close()
        logging.info(f"Filled form saved as {output_pdf_path}")
        print(f"Filled form saved as {output_pdf_path}")
    except Exception as e:
        logging.error(f"Failed to fill static PDF: {e}")

def create_output_folder(base_folder, email_address):
    """Creates a unique folder for each email address under the base folder."""
    email_folder = os.path.join(base_folder, email_address)

    if not os.path.exists(email_folder):
        os.makedirs(email_folder)
        logging.info(f"Created folder: {email_folder}")
    else:
        logging.info(f"Folder already exists: {email_folder}")

    return email_folder

def generate_output_file_path(folder, form_name, timestamp=None):
    """Generates a unique output file path."""
    if timestamp:
        return os.path.join(folder, f"filled_{form_name}_{timestamp}.pdf")
    else:
        return os.path.join(folder, f"filled_{form_name}.pdf")

def process_form(form_name, df):
    """Processes a single form for all rows in the DataFrame."""
    form_config = config.FORMS_CONFIG.get(form_name)
    if not form_config:
        logging.error(f"Form configuration for '{form_name}' not found.")
        return

    static_pdf_path = os.path.join(config.FOLDER_PATH, form_config["STATIC_PDF_PATH"])
    mapping_file_path = os.path.join(config.FOLDER_PATH, form_config["MAPPING_FILE_PATH"])
    output_folder_base = os.path.join(config.FOLDER_PATH, "filled")

    # Ensure the base output folder exists
    os.makedirs(output_folder_base, exist_ok=True)

    # Load the field mapping for the selected form
    with open(mapping_file_path, 'r', encoding='utf-8') as json_file:
        field_mapping = json.load(json_file)

    # Process each row of the DataFrame
    for index, row in df.iterrows():
        email_address = row["S2.5. Email Address"]
        #timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        timestamp_str = datetime.now().strftime("%Y%m%d")

        # Create a unique subfolder for each email address under the filled folder
        email_folder = create_output_folder(output_folder_base, email_address)

        # Check if forms already exist in the folder and append timestamp if necessary
        existing_files = os.listdir(email_folder)
        if existing_files:
            output_pdf_path = generate_output_file_path(email_folder, form_name, timestamp_str)
        else:
            output_pdf_path = generate_output_file_path(email_folder, form_name)

        # Fill the PDF form with data from the current row
        data = row.to_dict()  # Convert row to dictionary
        fill_static_pdf(static_pdf_path, output_pdf_path, field_mapping, data)

def main(fill_option=None):
    """Main function to load data and fill PDF forms."""

    # Use argparse only when running in non-interactive mode (e.g., terminal)
    if fill_option is None:
        parser = argparse.ArgumentParser(description="Fill PDF forms based on Google Sheets data.")

        parser.add_argument("--fill", type=str,
                            default=config.DEFAULT_FILL,
                            help="Specify which forms to fill (e.g., 'all', '1145', '9089', '140'). Default is set in config.py.")

        args = parser.parse_args()

        fill_option = args.fill  # Get --fill argument or default value

    # Mount Google Drive (for Colab environments)
    drive.mount('/content/drive')

    # Load data from Google Sheets
    df = load_google_sheet_data(config.GOOGLE_SHEET_ID, config.GOOGLE_CREDENTIALS_PATH)
    if df is None:
        logging.error("Exiting due to data loading failure.")
        return

    # Determine which forms to process based on the --fill argument or default value in config.py
    if fill_option == "all":
        for form_name in config.FORMS_CONFIG.keys():
            process_form(form_name, df)
    elif fill_option in config.FORMS_CONFIG.keys():
        process_form(fill_option, df)
    else:
        logging.error(f"Invalid value for --fill: {fill_option}. Must be one of 'all', '1145', '9089', or '140'.")

if __name__ == "__main__":

    try:
      get_ipython()  # Check if running in an interactive environment (like Colab or Jupyter Notebook)
      main(fill_option=config.DEFAULT_FILL)  # Use default value from config.py when running interactively.

    except NameError:
      main()  # Run normally with argparse when executed from terminal.
